<a href="https://colab.research.google.com/github/mohammedhemed77/NN-zero-to-hero-personal-practice/blob/master/micrograd/micrograd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plot 
# add this commit message to it
# add another message 
# add another message 

micro_grad Engine class

In [ ]:
class Value :
 def __init__(self, data ,_childrwn= (), op='' ):
    self.data = data
    print("hello world")